In [89]:
from functools import reduce

with open('input.txt') as f:
    init_stream =  f.read()


stream = init_stream * 1000

pieces = [
    [0b0011110],

    [0b0001000,
    0b0011100,
    0b0001000],
    
    [0b0000100,
    0b0000100,
    0b0011100],
    
    [0b0010000,
    0b0010000,
    0b0010000,
    0b0010000],
    
    [0b0011000,
    0b0011000]
]

tower = [0b1111111]

def give_row_str(row):
    return f'{row:07b}'

def print_row(row):
    print(give_row_str(row).replace('1', '#').replace('0', '.'))

def print_piece(piece):
    s = ''
    for row in piece:
        s += give_row_str(row) + '\n'
    print(s.replace('1', '#').replace('0', '.'), end='')

def print_tower(tower=tower):
    s = ''
    for row in tower[::-1]:
        s += give_row_str(row) + '\n'
    print(s.replace('1', '#').replace('0', '.'), end='')

# https://www.geeksforgeeks.org/python-bitwise-or-among-list-elements/
def check_right_edge(piece):
    res = reduce(lambda x, y: x | y, piece)
    return (res & 0b0000001) > 0

def check_left_edge(piece):
    res = reduce(lambda x, y: x | y, piece)
    return (res & 0b1000000) > 0

def get_drop_pos(piece, steps):
    piece = piece.copy()
    for step in steps:
        if step == '>' and not check_right_edge(piece):
            for i in range(len(piece)):
                piece[i] = piece[i] >> 1
        elif step == '<' and not check_left_edge(piece):
            for i in range(len(piece)):
                piece[i] = piece[i] << 1
    return piece

def compare_overlap(drop_piece, tower):
    # drop piece and tower must have same number of rows
    for x, y in zip(drop_piece, tower):
        if x & y > 0:
            return True
    return False
    # drop_piece_red = reduce(lambda x, y: x | y, drop_piece)
    # tower_red = reduce(lambda x, y: x | y, tower)
    # return (drop_piece_red | tower_red) > 0



In [66]:
piece_i = 0
stream_i = 0
blocks = 0

while True:
    # get drop pos
    blocks += 1
    drop_piece = get_drop_pos(pieces[piece_i], stream[stream_i: stream_i + 4])

    stream_i += 4
    if len(tower) == 1:
        tower.extend(drop_piece[::-1])

    else:
        pos = 0
        while True:
            drop_piece_l = len(drop_piece)
            # simulate pos
            sim_pos = pos - 1
            # test down one, deep down would show how much of the piece is exposed
            tmp_submerged = min(abs(sim_pos), drop_piece_l)
            if abs(sim_pos) <= drop_piece_l:
                overlap_down = compare_overlap(drop_piece[-tmp_submerged:], tower[-tmp_submerged:][::-1])
            else:
                overlap_down = compare_overlap(drop_piece[-tmp_submerged:], tower[sim_pos:sim_pos+tmp_submerged][::-1])
                # print(overlap_down, pos, tmp_submerged, drop_piece_l)
                # print_piece(drop_piece[-tmp_submerged:])
                # # print_piece(tower[-tmp_submerged:][::-1])
                # print()
                # print_piece(tower[pos-1:pos-1+tmp_submerged][::-1])

            if overlap_down:
                break

            pos -= 1
            # added this here because i have been constantly using the prior tmp_piece...
            tmp_piece = drop_piece
            # shift left or right
            if stream[stream_i] == '<':
                if not check_left_edge(drop_piece):
                    tmp_piece = [x << 1 for x in drop_piece]
            elif stream[stream_i] == '>':
                if not check_right_edge(drop_piece):
                    tmp_piece = [x >> 1 for x in drop_piece]
            stream_i += 1
            if abs(pos) <= drop_piece_l:
                overlap_hori = compare_overlap(tmp_piece[-tmp_submerged:], tower[-tmp_submerged:][::-1])
            else:
                overlap_hori = compare_overlap(tmp_piece[-tmp_submerged:], tower[pos:pos+tmp_submerged][::-1])
            if overlap_hori:
                # change from break to continue, even if cannot overlap, will not break here
                continue
            drop_piece = tmp_piece


            # if move down, then increment pos and deep, deep has max val depending on the height of the piece
        if pos == 0:
            tower.extend(drop_piece[::-1])
            # print_piece(drop_piece)
            # print()
        else:
            submerged = min(abs(pos), drop_piece_l)
            piece_deep = drop_piece[-submerged:]
            piece_not_deep = drop_piece[:-submerged]

            # find overlapping segment
            # if pos + submerged == 0:
            #     tower_deep = tower[pos:]
            # else:
            #     tower_deep = tower[pos:pos+submerged]
            if abs(pos) <= drop_piece_l:
                tower_deep = tower[-submerged:]
            else:
                tower_deep = tower[pos:pos+submerged]
                # print('Hi')
            
            # print(pos, submerged)
            # print_piece(tower_deep)
            # print_piece(piece_deep)
            # print_piece(piece_not_deep)

            # print(submerged, piece_deep, piece_not_deep, tower_deep)
            # somehow having piece_deep[::-1] makes algo work...

            # had to change range to reversed, and tower_deep to tower_deep[::-1]
            for i, pd, td in zip(reversed(range(pos, pos+submerged)), tower_deep[::-1], piece_deep):
                # print_row(pd)
                # print_row(td)
                # print()
                comb = pd | td

                if pd & td:
                    print(blocks, pd, td)
                tower[i] = comb
                
            # append not deep piece
            if piece_not_deep:
                tower.extend(piece_not_deep[::-1])
    piece_i = (piece_i + 1) % (len(pieces))

    # if blocks == 38:
    #     print_piece(drop_piece)
    #     print()
    #     print(pos, submerged)
    #     print()
    #     print_piece(tower_deep[::-1])
    #     print()
    #     print_piece(piece_deep)
    #     print()
    #     print_piece(piece_not_deep)
    if blocks >= 2022:
        break


In [67]:
len(tower) - 1

3068

In [62]:
len(init_stream)

40

In [90]:
# part 2
piece_i = 0
stream_i = 0
blocks = 0

tower = [0b1111111]

stream_used_5_blocks = []
height_5_blocks = []

while True:
    blocks += 1
    if blocks % 5 == 1:
        start_stream_i = stream_i
        start_height = len(tower)

    # get drop pos
    drop_piece = get_drop_pos(pieces[piece_i], stream[stream_i: stream_i + 4])

    stream_i += 4
    if len(tower) == 1:
        tower.extend(drop_piece[::-1])
    else:
        pos = 0
        while True:
            drop_piece_l = len(drop_piece)
            # simulate pos
            sim_pos = pos - 1
            # test down one, deep down would show how much of the piece is exposed
            tmp_submerged = min(abs(sim_pos), drop_piece_l)
            if abs(sim_pos) <= drop_piece_l:
                overlap_down = compare_overlap(drop_piece[-tmp_submerged:], tower[-tmp_submerged:][::-1])
            else:
                overlap_down = compare_overlap(drop_piece[-tmp_submerged:], tower[sim_pos:sim_pos+tmp_submerged][::-1])
            if overlap_down:
                break

            pos -= 1
            # added this here because i have been constantly using the prior tmp_piece...
            tmp_piece = drop_piece
            # shift left or right
            if stream[stream_i] == '<':
                if not check_left_edge(drop_piece):
                    tmp_piece = [x << 1 for x in drop_piece]
            elif stream[stream_i] == '>':
                if not check_right_edge(drop_piece):
                    tmp_piece = [x >> 1 for x in drop_piece]
            stream_i += 1
            if abs(pos) <= drop_piece_l:
                overlap_hori = compare_overlap(tmp_piece[-tmp_submerged:], tower[-tmp_submerged:][::-1])
            else:
                overlap_hori = compare_overlap(tmp_piece[-tmp_submerged:], tower[pos:pos+tmp_submerged][::-1])
            if overlap_hori:
                # change from break to continue, even if cannot overlap, will not break here
                continue
            drop_piece = tmp_piece


            # if move down, then increment pos and deep, deep has max val depending on the height of the piece
        if pos == 0:
            tower.extend(drop_piece[::-1])
        else:
            submerged = min(abs(pos), drop_piece_l)
            piece_deep = drop_piece[-submerged:]
            piece_not_deep = drop_piece[:-submerged]

            if abs(pos) <= drop_piece_l:
                tower_deep = tower[-submerged:]
            else:
                tower_deep = tower[pos:pos+submerged]
            # somehow having piece_deep[::-1] makes algo work...
            # had to change range to reversed, and tower_deep to tower_deep[::-1]
            for i, pd, td in zip(reversed(range(pos, pos+submerged)), tower_deep[::-1], piece_deep):
                comb = pd | td
                tower[i] = comb
               
            # append not deep piece
            if piece_not_deep:
                tower.extend(piece_not_deep[::-1])
    piece_i = (piece_i + 1) % (len(pieces))

    if blocks % 5 == 0:
        stream_used_5_blocks.append(stream_i - start_stream_i)
        height_5_blocks.append(len(tower) - start_height)

    if blocks >= 10000:
        break

len(tower) - 1

15445

In [91]:
print(height_5_blocks)

[8, 7, 10, 11, 13, 9, 6, 8, 7, 2, 6, 7, 9, 9, 12, 5, 8, 7, 7, 11, 8, 12, 7, 9, 7, 8, 10, 7, 10, 8, 9, 7, 4, 7, 8, 6, 4, 8, 6, 11, 6, 8, 9, 9, 6, 7, 8, 9, 9, 6, 8, 8, 6, 8, 9, 9, 6, 8, 7, 7, 8, 7, 4, 10, 13, 6, 6, 7, 6, 5, 7, 6, 5, 6, 7, 9, 11, 6, 6, 9, 11, 5, 5, 10, 6, 7, 6, 6, 9, 6, 11, 4, 10, 10, 10, 6, 9, 9, 9, 11, 7, 7, 7, 6, 10, 4, 5, 6, 11, 8, 11, 10, 7, 6, 7, 9, 10, 9, 7, 9, 7, 7, 9, 13, 8, 7, 9, 9, 9, 9, 6, 5, 4, 7, 7, 6, 7, 7, 6, 8, 8, 6, 7, 9, 8, 7, 6, 7, 9, 8, 7, 8, 2, 8, 8, 9, 9, 3, 8, 7, 8, 7, 8, 7, 5, 5, 6, 9, 7, 8, 4, 9, 8, 6, 7, 9, 9, 8, 9, 9, 4, 8, 9, 6, 11, 7, 7, 9, 10, 6, 7, 8, 8, 9, 5, 6, 10, 5, 8, 7, 7, 7, 7, 8, 10, 8, 7, 7, 12, 7, 6, 7, 11, 9, 5, 8, 6, 8, 7, 8, 3, 7, 7, 7, 8, 5, 9, 10, 13, 7, 9, 8, 10, 7, 9, 7, 6, 8, 7, 7, 10, 10, 6, 8, 5, 6, 9, 10, 9, 8, 8, 6, 10, 8, 6, 11, 7, 6, 10, 9, 11, 7, 7, 12, 7, 11, 9, 9, 11, 10, 6, 8, 7, 9, 9, 9, 7, 9, 9, 8, 7, 10, 7, 7, 6, 11, 3, 7, 7, 11, 12, 9, 8, 6, 9, 7, 6, 10, 10, 8, 9, 8, 5, 7, 8, 6, 9, 5, 8, 5, 8, 6, 9, 8, 7, 7, 

In [92]:
first_20 = [8, 7, 10, 11, 13, 9, 6, 8, 7, 2, 6, 7, 9, 9, 12, 5, 8, 7, 7, 11]
seq = [8, 12, 7, 9, 7, 8, 10, 7, 10, 8, 9, 7, 4, 7, 8, 6, 4, 8, 6, 11, 6, 8, 9, 9, 6, 7, 8, 9, 9, 6, 8, 8, 6, 8, 9, 9, 6, 8, 7, 7, 8, 7, 4, 10, 13, 6, 6, 7, 6, 5, 7, 6, 5, 6, 7, 9, 11, 6, 6, 9, 11, 5, 5, 10, 6, 7, 6, 6, 9, 6, 11, 4, 10, 10, 10, 6, 9, 9, 9, 11, 7, 7, 7, 6, 10, 4, 5, 6, 11, 8, 11, 10, 7, 6, 7, 9, 10, 9, 7, 9, 7, 7, 9, 13, 8, 7, 9, 9, 9, 9, 6, 5, 4, 7, 7, 6, 7, 7, 6, 8, 8, 6, 7, 9, 8, 7, 6, 7, 9, 8, 7, 8, 2, 8, 8, 9, 9, 3, 8, 7, 8, 7, 8, 7, 5, 5, 6, 9, 7, 8, 4, 9, 8, 6, 7, 9, 9, 8, 9, 9, 4, 8, 9, 6, 11, 7, 7, 9, 10, 6, 7, 8, 8, 9, 5, 6, 10, 5, 8, 7, 7, 7, 7, 8, 10, 8, 7, 7, 12, 7, 6, 7, 11, 9, 5, 8, 6, 8, 7, 8, 3, 7, 7, 7, 8, 5, 9, 10, 13, 7, 9, 8, 10, 7, 9, 7, 6, 8, 7, 7, 10, 10, 6, 8, 5, 6, 9, 10, 9, 8, 8, 6, 10, 8, 6, 11, 7, 6, 10, 9, 11, 7, 7, 12, 7, 11, 9, 9, 11, 10, 6, 8, 7, 9, 9, 9, 7, 9, 9, 8, 7, 10, 7, 7, 6, 11, 3, 7, 7, 11, 12, 9, 8, 6, 9, 7, 6, 10, 10, 8, 9, 8, 5, 7, 8, 6, 9, 5, 8, 5, 8, 6, 9, 8, 7, 7, 8, 7, 7, 8, 8, 6, 7, 9, 11, 7, 6, 9, 9, 6, 6, 7, 7, 10, 9, 7, 6, 8, 9, 8, 8, 8, 4, 7, 9, 8, 3, 11, 6, 10, 9, 9, 5, 7, 6, 8, 13, 6, 10, 8]

In [93]:
len(seq)

340

In [96]:
(1000000000000 - len(first_20) * 5) // (len(seq) * 5)

588235294

In [100]:
(1000000000000 - len(first_20) * 5) % (len(seq) * 5)

100

In [99]:
588235294 * len(seq) * 5

999999999800

In [103]:
# // 5 since 1 
sum(first_20) + 588235294 * sum(seq) + sum(seq[:100 // 5])

1542941176480

In [88]:
print(height_5_blocks)

sum([11, 7, 8, 9, 6, 6, 6]) * 1000000000000 // (7 * 5)

[9, 8, 8, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 11, 7, 8, 9, 6, 6, 6, 

1514285714285

In [2]:
# check overlap
# struct for each to define the bottom, and the coordinates from a pivot

'''
If bottom - Check bottom whether anything is in the way
If left - perform overlap check
If right - perform overlap check
'''



'\nIf bottom - Check bottom whether anything is in the way\nIf left - perform overlap check\nIf right - perform overlap check\n'

In [17]:
# if want to check no overlap, check if a & b is false
a = 0b0000001 
a & 0b1111101

1

In [25]:
(a << 1) & 0b1111101 # ok = false

0

In [28]:
[0b1000010, 0b0101000, 0b10100010, 0b1000011]

[66, 40, 162, 67]